https://www.kaggle.com/datasets/jessicali9530/lfw-dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install opencv-python
!pip install insightface
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 17.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 44.3 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp311-cp311-linux_x86_64.whl size=1064780 sha256=3105aa644fa56d17d34bb39158c370240b37f343c2ff2e59447b2cdd0730bfbd
  Stored in directory: /root/.cache/pip/wheels/27/d8/22/f52d858d16cd06e7b2e6aad34a1777dcfaf000be833bbf8146
Successfully built insightface
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.7 MB/s eta 0:00:00


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from insightface.app import FaceAnalysis
import os
import cv2
import numpy as np
import pandas as pd
from itertools import combinations
from tqdm import tqdm

# --- Configuration ---
IMAGE_ROOT = "/content/drive/MyDrive/lfw-deepfunneled"
CSV_PATH = "/content/drive/MyDrive/peopleDevTest.csv"
THRESHOLD = 0.6

# Load ArcFace model
face_app = FaceAnalysis(name='buffalo_l')
face_app.prepare(ctx_id=0)

# Load CSV with names
df = pd.read_csv(CSV_PATH)

def get_embedding(image_path):
    full_path = os.path.join(IMAGE_ROOT, image_path)
    img = cv2.imread(full_path)
    if img is None:
        return None
    faces = face_app.get(img)
    return faces[0].embedding if faces else None

def evaluate(df):
    y_true = []
    y_pred = []

    print("Evaluating same-person pairs...")
    for _, row in tqdm(df.iterrows(), total=len(df)):
        person = row['name']
        person_dir = os.path.join(IMAGE_ROOT, person)
        if not os.path.isdir(person_dir):
            continue

        images = sorted(os.listdir(person_dir))
        if len(images) < 2:
            continue

        images = images[:2]
        pairs = list(combinations(images, 2))
        for img1, img2 in pairs:
            emb1 = get_embedding(os.path.join(person, img1))
            emb2 = get_embedding(os.path.join(person, img2))
            if emb1 is None or emb2 is None:
                continue

            similarity = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
            y_true.append(1)
            y_pred.append(1 if similarity > THRESHOLD else 0)
            print(f"[SAME] {person}: {img1} vs {img2} → Sim: {similarity:.3f} → {'Match' if similarity > THRESHOLD else 'No Match'}")

    print("\nEvaluating one-to-one different-person pairs (single image only)...")
    single_image_people = []
    for _, row in df.iterrows():
        person = row['name']
        person_dir = os.path.join(IMAGE_ROOT, person)
        if not os.path.isdir(person_dir):
            continue
        images = sorted(os.listdir(person_dir))
        if len(images) == 1:
            single_image_people.append((person, images[0]))

    # Make sure we have an even number of people for pairing
    if len(single_image_people) % 2 != 0:
        single_image_people = single_image_people[:-1]

    for i in range(0, len(single_image_people), 2):
        (p1, img1) = single_image_people[i]
        (p2, img2) = single_image_people[i+1]

        emb1 = get_embedding(os.path.join(p1, img1))
        emb2 = get_embedding(os.path.join(p2, img2))
        if emb1 is None or emb2 is None:
            continue

        similarity = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
        y_true.append(0)
        y_pred.append(1 if similarity > THRESHOLD else 0)
        print(f"[DIFFERENT] {p1} vs {p2} → Sim: {similarity:.3f} → {'Match' if similarity > THRESHOLD else 'No Match'}")

    # --- Summary Metrics ---
    print("\n--- Confusion Matrix ---")
    print(confusion_matrix(y_true, y_pred, labels=[1, 0]))  # [TP, FN], [FP, TN]

    print("\n--- Classification Report ---")
    print(classification_report(y_true, y_pred, target_names=["Different-Person (Neg)", "Same-Person (Pos)"]))

# --- Run ---
evaluate(df)


/usr/local/lib/python3.11/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:118: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)
Evaluating s

  0%|          | 7/1711 [00:03<12:43,  2.23it/s]

[SAME] Abdullah_Gul: Abdullah_Gul_0001.jpg vs Abdullah_Gul_0002.jpg → Sim: 0.619 → Match


  0%|          | 8/1711 [00:05<23:05,  1.23it/s]

...

 99%|█████████▉| 1699/1711 [17:47<00:05,  2.25it/s]

[SAME] Yoriko_Kawaguchi: Yoriko_Kawaguchi_0001.jpg vs Yoriko_Kawaguchi_0002.jpg → Sim: 0.660 → Match


100%|██████████| 1711/1711 [17:49<00:00,  1.60it/s]

[SAME] Zoran_Djindjic: Zoran_Djindjic_0001.jpg vs Zoran_Djindjic_0002.jpg → Sim: 0.797 → Match

Evaluating one-to-one different-person pairs (single image only)...


[DIFFERENT] AJ_Lamas vs Aaron_Guiel → Sim: 0.065 → No Match
[DIFFERENT] Aaron_Tippin vs Abba_Eban → Sim: -0.078 → No Match
[DIFFERENT] Abdul_Majeed_Shobokshi vs Abdulaziz_Kamilov → Sim: 0.090 → No Match
[DIFFERENT] Abel_Aguilar vs Adam_Ant → Sim: 0.014 → No Match
[DIFFERENT] Adam_Kennedy vs Adrian_Annus → Sim: 0.018 → No Match
[DIFFERENT] Adrian_Murrell vs Adriana_Lima → Sim: -0.022 → No Match
[DIFFERENT] Adriana_Perez_Navarro vs Agbani_Darego → Sim: 0.012 → No Match
[DIFFERENT] Agnelo_Queiroz vs Ahmed_Ghazi → Sim: -0.030 → No Match
[DIFFERENT] Aileen_Riggin_Soule vs Ain_Seppik → Sim: -0.082 → No Match
[DIFFERENT] Al_Cardenas vs Alan_Dershowitz → Sim: 0.084 → No Match
[DIFFERENT] Alan_Stonecipher vs Alan_Zemaitis → Sim: -0.022 → No Match
[DIFFERENT] Alecos_Markides vs Alejandro_Fernandez → Sim: 0.060 → No Match
[DIFFERENT] Aleksander_Voloshin vs Alessandro_Nesta → Sim: 0.016 → No Match
[DIFFERENT] Alex_Cabrera vs Alex_Corretja → Sim: 0.060 → No Match
[DIFFERENT] Alex_Zanardi vs Alexand